# Generating model inputs from raw audio

Generating spectrograms and then saving those images as _images_ loses much of the information stored in the spectrogram input data. The spectrogram input is a complex-valued matrix D, consisting of magnitude and phase of frequency bin f at frame t. A plotted spectrogram shows time on the x-axis, frequency on the y-axis, with amplitude/intensity/decibels as color.  The data here have a minimum sample rate of 50KHz, signifying 50,000 samples per second for the :30 clip. The frequency and time binning done to generate a spectrogram array groups these 1.5M+ samples into windows of specified length. 

The 12.5-25KHz frequency band encompasses most of the whistle vocalizations of the target species; their clicks are broadband and should be visible in this frequency as well.

If transposing the resulting image to a .png and reimporting, the resulting matrix dimensions represent pixels (based on physical size of export), and resulting matrix values represent a color on a different scale than audio intensity. Very nuanced standardization of spectrograms, cmaps, etc may yield a comparable input for CNN; but given the significant smoothing, binning, resampling, frequency ranges, and other parameters that go into generating each spectrogram matrix, I think the information loss is too great, or at least too unquantifiable.

To "feed spectrograms to CNN", what we want is the true spectrogram matrix, not the matrix of the .png (or other image format) rendering of the spectrogram matrix. 

---
Parameters used herein, based on marine mammal and underwater acoustic literature review:
* sr=50KHz --> all data are either 50k or 64k; resample the 64k files to 50k so working with equal times per frame
* n_fft=4096; downsized to accommodate file size issues. Ideal was n_fft>=8192 (based on methods from Thomas et al 2019, and applying to the unique attributes of _S. longirostris_ vocalizations)
* win_length = n_fft
* hop_length = n_fft/2; downsized to accommodate file size issues. Ideal was win_length/4 (default)
* window: Hann window
    
---
  
This notebook generates spectrogram matrices for training and test data with Librosa's stft function.



In [2]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import os
import wave
import librosa
import librosa.display
import IPython.display as ipd
import pickle

## Step 1. set aside holdout set
This set will remain separate from regular train/test data for final validation.  
500 positive files, 1538 negative files

In [3]:
#relocate positives and negatives from existing folders to holdout folders
def define_holdout(n_pos, pos_origin, pos_dest, n_neg, neg_origin, neg_dest):
    import shutil
    
    #positives
    for i in range(n_pos):
        shutil.move(pos_origin + np.random.choice(os.listdir(pos_origin)), pos_dest)
        
    #negatives
    for j in range(n_neg):
        shutil.move(neg_origin + np.random.choice(os.listdir(neg_origin)), neg_dest)
        
    print(f'# positives in holdout: {len(os.listdir(pos_dest))}')
    print(f'# negatives in holdout: {len(os.listdir(neg_dest))}')
    print(f'# pos for tts: {len(os.listdir(pos_origin))}')
    print(f'# neg for tts: {len(os.listdir(neg_origin))}')

In [4]:
pos_origin = '../scratch_data/yes_dolphin/'
pos_dest = '../scratch_data/holdout/positives'
neg_origin = '../scratch_data/no_dolphin/'
neg_dest = '../scratch_data/holdout/negatives'

# define_holdout(500, pos_origin, pos_dest, 1538, neg_origin, neg_dest)

# positives in holdout: 501
# negatives in holdout: 1538
# pos for tts: 5588
# neg for tts: 16766

## Step 2. Create spectrograms for tts
* Pos and Neg files currently only differentiated by directory; attach label (Yes/No) here
* Trim arrays to only frequencies over 10KHz; critical range 12.5KHz-25KHz
    * If data size still unweildy, trim to 12KHz
    * rows to trim indicated by fft_freq results
* Save spectrogram matrices and labels in list(s), save list via pickle or librosa cache?

In [5]:
#index of frequency bins that are below threshold depend on features of stft
#function to define lower threshold

def thresh(array, thresh_hz):
    vals=[]
    for i in range(len(array)):
        vals.append(abs(thresh_hz-array[i]))
    return vals.index(min(vals)) #returns index of almost-thresh value in ascending list

In [6]:
freq_key = librosa.fft_frequencies(sr=50000, n_fft=4096)
thresh(freq_key, 10000)
#indicates trim stft array to [820:]

819

In [7]:
#redefine thresh here so can run as single cell
#(thresh fx called in listospects)
def thresh(array, thresh_hz):
    vals=[]
    for i in range(len(array)):
        vals.append(abs(thresh_hz-array[i]))
    return vals.index(min(vals))


def listospects(in_path, samp_rate, n_fft, win_length, window, pos_neg, low_thresh, ex_path):
    import pickle
    
    freq_key = librosa.fft_frequencies(sr=samp_rate, n_fft=n_fft)
    z = thresh(freq_key, low_thresh)
    
    
    files = os.listdir(in_path)
    spectrolist = []
    count = 1
    
    for i in files:
        y, sr = librosa.load(in_path + i, sr=samp_rate)
        S = np.abs(librosa.stft(y, n_fft=n_fft, win_length=win_length, hop_length=int(n_fft/2),
                               window=window))
        trim = S[z+1:] #only keep values above lower freq threshold
        
        chunk = S[0:1229:2]  #downsample freq axis for compression
        
        spectrolist.append(chunk)
        
        if len(spectrolist) % 1000 == 0:
            print(len(spectrolist))
            
        if len(spectrolist) % 3000 == 0: #save results so far and start anew (wary of pickle size limits)
            with open(f'{ex_path}tts_{count}_{pos_neg}.pkl', mode = 'wb') as pickle_out:
                pickle.dump(spectrolist, pickle_out)
            count += 1
            spectrolist = []
            
    #export whatever's in list when pau        
    if len(spectrolist) > 0:
        with open(f'{ex_path}tts_{count}_{pos_neg}.pkl', mode = 'wb') as pickle_out:
            pickle.dump(spectrolist, pickle_out)
            
    print("thanks for all the fish")

In [8]:
#positive IDs first

in_path = '../scratch_data/yes_dolphin/'
ex_path = '../scratch_data/tts_arrays/'
yes_tts = listospects(in_path=in_path, samp_rate=50000, n_fft=4096, win_length=4096,
                      window=signal.windows.hann, pos_neg='pos', low_thresh=10000, ex_path=ex_path)

1000
2000
3000
1000
2000
thanks for all the fish


In [9]:
#negative IDS. 4:30 not bad!
in_path = '../scratch_data/no_dolphin/'
ex_path = '../scratch_data/tts_arrays/'
yes_tts = listospects(in_path=in_path, samp_rate=50000, n_fft=4096, win_length=4096,
                      window=signal.windows.hann, pos_neg='neg', low_thresh=10000, ex_path=ex_path)

1000
2000
3000
1000
2000
3000
1000
2000
3000
1000
2000
3000
1000
2000
3000
1000
thanks for all the fish


In [10]:
with open ('../scratch_data/tts_arrays/tts_4_neg.pkl', mode = 'rb') as pickle_in:
    check = pickle.load(pickle_in)

In [17]:
print(len(check))
print(check[4].shape)
check[4]

3000
(615, 740)


array([[0.04334272, 0.10069173, 0.09125934, ..., 0.2773754 , 0.3506636 ,
        0.3499492 ],
       [0.07013685, 0.10394271, 0.11942144, ..., 0.09624764, 0.09879693,
        0.10170203],
       [1.0786785 , 0.37674418, 0.01665294, ..., 0.34638163, 0.3596837 ,
        0.3092308 ],
       ...,
       [0.26128706, 0.2993274 , 0.428127  , ..., 0.2639781 , 0.19433732,
        0.37926418],
       [0.60569626, 0.21978666, 0.3518643 , ..., 0.7123921 , 0.5948586 ,
        0.36077625],
       [0.26761234, 0.30750263, 0.26671088, ..., 0.1334184 , 0.51322204,
        0.3484277 ]], dtype=float32)

In [16]:
import sys
sys.getsizeof(check)

27104

### Future work
The parameters I planned to start with based on consultations and literature review are different than those used here. Computing power and time were both limiting factors for the scope of this initial modeling, so most parameters have been downsampled. The resulting features for model input are much lower resolution than preconceived, but this will still be informative for future steps. And hey, there's a high chance what I deem important features as a person trying to make sense of these audio and visual representations are not actually critical for machine learning.  The following specs are either what I wanted to implement here if I had better resources on hand, or what I intend to implement in the future:  

* n_fft
    * Future goal / next step: use multiple n_fft windows: train separate NNs on spectrograms of different n_ffts and compare performance, OR stack/interpolate into single spectrogram
    * n_fft=128 could best capture clicks (bursts and/or trains) 
        * 128/50000 = .00256 sec = 2560$\mu$s per FFT
        * fastest dolphin clicks (burst pulses) ~ 1750 clicks/sec = 570$\mu$s between clicks, each click 50-128 $\mu$s duration 
        * 2560$\mu$s window could capture several clicks; too short for whistles
    * more inclusive window, clicks and whistles: ~ sr/4 = 12500; n_fft=8192(.16 sec); n_fft=16384(.33s)
        * _S. longirostris_ whistle duration = 0.05-1.28s, avg .49s
        * don't want to exceed n_fft=16384 (next power of 2 > 32K) because matrix size getting out of hand?

* win_length: Future --> smaller values to better discriminate clicks

* hop_length
    * intent was win_length / 4, downsampled to win_length/2 above
    
* window
    * Future: compare model performance using Hann vs. Hamming
    * Blackman might be another window suitable for this audio but what do I know